# Exploration File
This project aims to build a semantic search engine using various NLP techniques. The dataset used is a csv file downloaded from kaggle titled "English Premier League - Match Commentary"

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
df = pd.read_csv("D:\\programs\\labs\\semantic-search-epl\\data\\23_24_match_details.csv")
df

,id,Home,Away,Date,Stadium,Attendance,Referee,events,summary
0,93323,Bournemouth,West Ham,2023-08-12,"Vitality Stadium, Bournemouth",NaN,Robert Jones,Hello and welcome to live coverage of the Prem...,Referee: Peter Bankes. Assistants: Dan Robatha...
1,93336,Man City,Newcastle,2023-08-19,"Etihad Stadium, Manchester",NaN,Robert Jones,Hello everyone and welcome to live text covera...,"Referee: Robert Jones. Assistants: Ian Hussin,..."
2,93343,Brentford,Crystal Palace,2023-08-26,"Gtech Community Stadium, Brentford",16997.0,Peter Bankes,Hello and welcome to the live commentary of th...,NaN
3,93344,Brighton,West Ham,2023-08-26,"American Express Stadium, Falmer",31508.0,Anthony Taylor,Hello and welcome to live coverage of the Prem...,Referee: Anthony Taylor. Assistants: Gary Besw...
4,93347,Everton,Wolves,2023-08-26,"Goodison Park, Liverpool",38851.0,Craig Pawson,Hello and welcome to live coverage of this Pre...,NaN
...,...,...,...,...,...,...,...,...,...
298,93647,Man City,Luton,2024-04-13,"Etihad Stadium, Manchester",53449.0,John Brooks,Hello and welcome to live coverage of the Prem...,Manchester City moved top of the Premier Leagu...
299,93641,Arsenal,Aston Villa,2024-04-14,"Emirates Stadium, London",60350.0,David Coote,Hello and welcome to our LIVE commentary of th...,Arsenal failed to take advantage of Liverpool’...
300,93650,West Ham,Fulham,2024-04-14,"London Stadium, London",62459.0,Stuart Attwell,"Hello, and welcome to live coverage of this fi...",Andreas Pereira proved the hero with a deadly ...
301,93646,Liverpool,Crystal Palace,2024-04-14,"Anfield, Liverpool",60090.0,Chris Kavanagh,Hello and welcome to our live coverage of the ...,Eberechi Eze dealt a huge blow to Liverpool’s ...
